In [220]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
import threading
import time
import math
import scipy as sp
from datetime import datetime

## Group Assignment
### Team Number: 08
### Team Member Names: Richard Meng, Avaansh Nanda, Devesh Kotak
### Team Strategy Chosen: Market Beat

We first start by reading the CSV file of tickers and processing the tickers as a list for easy iteration.

In [221]:
# read in the csv file and set the header to none, otherwise it will treat the first ticker as the header
tickers_csv = pd.read_csv("Tickers_Example.csv", header=None)
# turn the csv dataframe into a list
tickers = list(tickers_csv[0])
tickers

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

Below are some helper functions to help assist filtering our stock data using data threading. 

In [222]:
#  We will be filtering 40 tickers. Each thread will handle 5 tickers which download from yfinance simultaneously, speeding up the process.
START_DATE = "2024-10-01"
END_DATE = "2025-09-30"
tickers_data = {}
monthly_tickers_data = {}

# helper function to download stock data 
def download_yf(ticker_subset, start, end):
    # looping through each ticker and downloading the data from yfinance, and storing in a dict
    for ticker in ticker_subset:
        tickers_data[ticker] = yf.download(ticker, start, end, auto_adjust=False, progress=False)

# helper function to download MONTHLY stock data
def download_monthly_yf(ticker_subset, start, end):
    # looping through each ticker and downloading the data from yfinance, and storing in a dict
    for ticker in ticker_subset:
        monthly_tickers_data[ticker] = yf.download(ticker, start, end, auto_adjust=False, progress=False, interval="1mo")


# stored in a dictionary where each key pair value is the ticker, DataFrame of data
download_yf(tickers, START_DATE, END_DATE)
tickers

ERROR: 
1 Failed download:
ERROR: ['AGN']: YFTzMissingError('possibly delisted; no timezone found')
ERROR: 
1 Failed download:
ERROR: ['CELG']: YFTzMissingError('possibly delisted; no timezone found')
ERROR: 
1 Failed download:
ERROR: ['MON']: YFTzMissingError('possibly delisted; no timezone found')
ERROR: 
1 Failed download:
ERROR: ['RTN']: YFTzMissingError('possibly delisted; no timezone found')


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

Next, we begin filtering the stocks. We first must check that they are Canadian or American, which can be done by checking their currency. Then we check if the company is actually listed on its respective market. Lastly, we check the daily average volume (excluding months with less than 18 trading days) and exclude stocks with less than 5000 daily average volume.

In [223]:
# function to check if a singular ticker is valid, then add it to an array of results at its respective index
# this is the helper function for the thread
MIN_TRADING_DAYS = 18
MIN_VOLUME = 5000
VALID_CURRENCIES = ['USD','CAD']

def valid_ticker(ticker, results, index):
    try:
        # store the ticker as a yf.Ticker to find info on currency and sector
        yfTicker = yf.Ticker(ticker)
        # store data to get trading volume + loop through the dates to find number of trading days per month
        ticker_data = tickers_data[ticker]
        # use time.sleep so yFinance does not time out and limit our request rate due to threading
        time.sleep(2)
        # store the currency
        currency = yfTicker.fast_info.get('currency')
        # store the sector
        sector = yfTicker.info.get('sector')
        # again use time.sleep after each yFinance call so it does not limit our request rate
        time.sleep(2)
        # create a date range with the start of each month in range
        month_starts = pd.date_range(start=START_DATE, end=END_DATE, freq="MS")
        # create a date range with the end of each month in range
        month_ends = pd.date_range(start=START_DATE, end=END_DATE, freq="ME")
        # initialize current month index to 0 and num trading days equal to 0
        cur_month_idx = 0
        num_trading_days = 0
        # keep track of the tickers total volume and number of total trading days to find avg daily volume
        total_vol = 0
        total_trading_days = 0
        # initialize date index
        date_idx = 0
        dates = ticker_data.index
        # loop through each date in the ticker data
        while date_idx < len(dates) and cur_month_idx < len(month_starts):
            date = dates[date_idx]
           # if the date is less than the end date of current month it is in
            if date <= month_ends[cur_month_idx]:
                # first add one to the number of trading days
                num_trading_days += 1
                total_trading_days += 1
                # if it is greater than 18, now we check the daily average volume
                #    if its less than 5000 don't add this stock to valid stocks.
                # otherwise, if the number of trading days isn't greater than 18
                #    continue on as then we'll exclude it or wait until it goes above 18
                if num_trading_days >= MIN_TRADING_DAYS:
                    ticker_month_data = yf.download(ticker, start=month_starts[cur_month_idx], end=month_ends[cur_month_idx], auto_adjust=False, progress=False)
                    # again use time.sleep to allow our yFinance requests to go through
                    time.sleep(2)
                    # add to total volume
                    total_vol += ticker_month_data["Volume"].sum().iloc[0]
                    # find the current index of the last date, side="right" gives us the index after the last day of the month's index
                    date_idx = np.searchsorted(dates, month_ends[cur_month_idx], side="right")

            # now we have surpassed the month end and will go to the next month       
            else:
                cur_month_idx += 1
                num_trading_days = 0

            date_idx+=1
        # calculate average daily volume
        avg_daily_vol = total_vol / total_trading_days if total_trading_days > 0 else 0
        # if the average daily volume is less than the minimum or the currency is invalid, don't add the ticker (store it as None).
        if avg_daily_vol<MIN_VOLUME or not(currency in VALID_CURRENCIES):
            results[index] = None
            print(f"removed {ticker}")
        # otherwise, if it's valid, then add the key pair ticker, (sector, currency) to valid_tickers for easy access to a ticker's info.
        else:
            results[index] = (ticker, sector, currency)
    # the except will catch delisted stocks and exclude them from valid_tickers (store it as None)
    except Exception as e:
        print(f"{ticker} is delisted")
        results[index] = None

# function for threading to speed up the process
def filter_tickers(tickers):
    # initialize an list threads to store our threads and valid_tickers to get the return value of the valid_ticker function
    threads = []
    valid_tickers = [""] * len(tickers)

    # loop through the list of possible tickers and get the index and actual ticker
    for index, ticker in enumerate(tickers):
        # start a thread using the valid_ticker function and pass in its respective arguments
        t = threading.Thread(target=valid_ticker, args=(ticker, valid_tickers, index))
        # start the thread and append it to the threads list
        t.start()
        threads.append(t)

    # loop through each thread and join them
    for thread in threads:
        thread.join()
    # use a list comprehension to add to valid_tickers if the ticker is not None.
    return [v for v in valid_tickers if v]

# store the valid_tickers in a list
valid_tickers = filter_tickers(tickers)
# initalize a dictionary to store elements as ticker : (sector, currency)
ticker_dict = {}
# add all valid tickers to the dictionary
for ticker_tup in valid_tickers:
    ticker_dict[ticker_tup[0]] = ticker_tup[1::]

# then change valid_tickers to a list of only the tickers
valid_tickers = [t[0] for t in valid_tickers]

ERROR: $RTN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR: $MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR: $CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR: $AGN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


RTN is delisted
MON is delisted
CELG is delisted
AGN is delisted


Here is the list of all tickers that follow the rules and their respective information stored in a dictionary

In [224]:
print(len(ticker_dict.keys()), len(tickers))
ticker_dict

37 41


{'AAPL': ('Technology', 'USD'),
 'ABBV': ('Healthcare', 'USD'),
 'ABT': ('Healthcare', 'USD'),
 'ACN': ('Technology', 'USD'),
 'AIG': ('Financial Services', 'USD'),
 'AMZN': ('Consumer Cyclical', 'USD'),
 'AXP': ('Financial Services', 'USD'),
 'BA': ('Industrials', 'USD'),
 'BAC': ('Financial Services', 'USD'),
 'BB.TO': ('Technology', 'CAD'),
 'BIIB': ('Healthcare', 'USD'),
 'BK': ('Financial Services', 'USD'),
 'BLK': ('Financial Services', 'USD'),
 'BMY': ('Healthcare', 'USD'),
 'C': ('Financial Services', 'USD'),
 'CAT': ('Industrials', 'USD'),
 'CL': ('Consumer Defensive', 'USD'),
 'KO': ('Consumer Defensive', 'USD'),
 'LLY': ('Healthcare', 'USD'),
 'LMT': ('Industrials', 'USD'),
 'MO': ('Consumer Defensive', 'USD'),
 'MRK': ('Healthcare', 'USD'),
 'PEP': ('Consumer Defensive', 'USD'),
 'PFE': ('Healthcare', 'USD'),
 'PG': ('Consumer Defensive', 'USD'),
 'PM': ('Consumer Defensive', 'USD'),
 'PYPL': ('Financial Services', 'USD'),
 'QCOM': ('Technology', 'USD'),
 'RY.TO': ('Financi

Since we must choose 10-25 stocks, we will first rank the top 25 stocks using a metric or scoring system we came up with.

Each stock has a score: 
* $\ 0.5*Sharpe \ Ratio + 0.5*Alpha $

We chose this metric because the Sharpe Ratio provides the risk-adjusted return, while the Alpha indicates the return above the market. 

We chose the $\ Sharpe \ Ratio$ as we still would like to be slightly risk-averse and it provides us a strong indicator of how much a stock returns given its risk, over a long-term period. As for $\ Alpha$, this helps us with our goal, of market beat, by giving a stock's specific return above the market.

We will find the Sharpe Ratio of each stock by finding the returns of each stock, the risk-free rate given by t-bills (dependent on currency), and the standard deviation of each stock.

Then we will find the Alpha of each stock by finding the returns of each stock and the returns of its respective market (also dependent on currency)

Then we output the top 25 stocks in order from highest to least score out of the valid stocks.

In [225]:
DATA_START_DATE = "2022-06-01"
DATA_END_DATE = "2025-09-30"
download_monthly_yf(valid_tickers, start=DATA_START_DATE, end=DATA_END_DATE)


# helper function to get the average returns for the index 
def get_index_returns(index_ticker):
    # downloading monthly data from the index 
    index_data = yf.download(index_ticker, start=DATA_START_DATE, end=DATA_END_DATE, auto_adjust=False, progress=False, interval="1mo")
    index_returns = index_data["Close"].pct_change().dropna().mean().item() # .item() gets the acutal value we need from the series
    return index_returns

# function to retrieve risk free rates for respective markets (American/Canadian)
def get_risk_free_rates():
    # ^TNX - 10 year Treasury Yield, mostly risk free and commonly used to calculate risk-free rate 
    # From Bank Of Canada, current treasury bond yields for 1 year are 2.31% (as of November 12, 2025)
    american_bond_data = yf.download("^TNX", period="1d", auto_adjust=False, progress=False)  # get the most recent rate (most recent closing price)
    american_rf_rate = (tnx['Close'].iloc[-1] / 100).item() # divide by 100 to get decimal 
    canadian_rf_rate = 0.0231
    return (american_rf_rate, canadian_rf_rate)

RF_rates = get_risk_free_rates()
USD_RF = RF_rates[0]
CAD_RF = RF_rates[1]

# function to calculate the sharpe ratio of all the stocks 
def get_sharpe_ratios(tickers):
    sharpe_ratios = {} # dictionary of the sharpe ratios
    # looping through each stock and calculating its sharpe ratio
    for ticker in tickers:
        ticker_currency = ticker_dict[ticker][1]
        RF = USD_RF/12 if ticker_currency == "USD" else CAD_RF/12 # divide by 12 since we look at monthly data here
        returns = monthly_tickers_data[ticker]["Close"].pct_change().dropna()
        Rp = returns.mean() # calculating rate of return of the portfolio 
        std_portfolio = returns.std() # calcualting std of the portfolio 
        
        sharpe_ratio = ((Rp - RF) / std_portfolio) * math.sqrt(12) # multiplying by sqrt(12) to annualize sharpe ratio
        sharpe_ratios[ticker] = sharpe_ratio.item() # adding to the dictionary, .item() to convert from series
    
    return sharpe_ratios

# helper function to get the alpha values (returns vs market returns) of all the stocks 
def get_alpha(tickers):
    alpha_values = {} # dictionary to store ("Ticker": alpha_value) for each stock
    tsx_avg_returns = get_index_returns("^GSPTSE")
    sp_avg_returns = get_index_returns("^GSPC")
    index_avg_returns = None # will choose based on stock's currency
    for ticker in tickers:
        returns = monthly_tickers_data[ticker]["Close"].pct_change().dropna().mean()
        index_ticker = None # the index will either be TSX or S&P 500
        ticker_currency = ticker_dict[ticker][1]
        # if the ticker's currency is CAD, then the benchmark will be TSX, if USD, then S&P 500
        if ticker_currency == "CAD":
            index_avg_returns = tsx_avg_returns
        else:
            index_avg_returns = sp_avg_returns

        alpha = (returns - index_avg_returns) * 12 # * 12 to annualize
        
        alpha_values[ticker] = alpha.item() # adding to the dictionary, .item() to convert from series 

    return alpha_values

# getting the alpha values and sharpe ratios and converting to a list so we can sort 
alpha_values = get_alpha(valid_tickers)
sharpe_ratios = get_sharpe_ratios(valid_tickers)


SHARPE_WEIGHTING = 0.5
ALPHA_WEIGHTING = 0.5

# a dictionary of the total scores of each stock (sharpe + alpha) 
total_scores = {ticker: ALPHA_WEIGHTING*alpha_values[ticker] + SHARPE_WEIGHTING*sharpe_ratios[ticker] for ticker in valid_tickers}
sorted_tickers = sorted(total_scores.keys(), key=total_scores.get, reverse=True) # getting a list of the keys (tickers) of total_scores by score
sorted_scores = {ticker: total_scores[ticker] for ticker in sorted_tickers}

# function to get the top 17 stocks based on their score 
def get_top_stocks(n):
    top_stocks = {} # empty dictionary to store the top n stocks
    for i in range(0, n):
        ticker = sorted_tickers[i]
        score = sorted_scores[ticker]
        top_stocks[ticker] = score

    return top_stocks

top_stocks = get_top_stocks(25)
top_stocks



# sorting the dictionaries in ascending order 
#dict(sorted(alpha_values.items()))

# TODO: We need to factor in fees

# function to create a portfolio with the best 17 stocks based on 50% weighting in sharpe ratio, 50% in alpha.
#def create_portfolio():
 #   dict(sorted(alpha_values.items()))

# NEXT THINGS TO DO
# 1. Make a function which creates a portfolio with the best 17 stocks based on 50% weighting in sharpe ratio, 50% in alpha. - DONE

# 2. Factor in the restriction of MAX 40% can be in one industry <- might take most time - NEXT TIME
# 3. Create the optimal stock weightings using SciPi minimize() function (Richard will do this)
# 4. Once 1, 2,3. are working, expand it to a loop that goes through 10-25 stocks, then compares those portfolios to get the one with the highest return 


{'SHOP.TO': 0.8066124553252085,
 'BK': 0.711606043874371,
 'AXP': 0.5659402455828234,
 'CAT': 0.5494766354017158,
 'LLY': 0.4946913284268909,
 'C': 0.47601212042207214,
 'RY.TO': 0.41408677238737346,
 'AMZN': 0.40610901275362776,
 'BLK': 0.38989260933963965,
 'AAPL': 0.37497788959851786,
 'MO': 0.2947635367800783,
 'PM': 0.29293143630175716,
 'BAC': 0.2769995063384029,
 'AIG': 0.23943847369263532,
 'BA': 0.23593326736225453,
 'ABBV': 0.22788735904738136,
 'TD.TO': 0.19788298941627613,
 'BB.TO': 0.11849708180100561,
 'QCOM': 0.11040721428851083,
 'ABT': 0.053719376342556906,
 'TXN': 0.04524788692434739,
 'LMT': 0.00710133058876862,
 'USB': -0.020804527669624838,
 'UNP': -0.04207629976895705,
 'PYPL': -0.06709705874516893}

Below is a sample of how optimal weightings will work

In [226]:
initial_weightings = [1/25]*25
top_tickers = list(top_stocks.keys())
CAD_INITIAL_INVESTMENT = 1_000_000
stock_shares = {}

EXCHANGE_RATE = yf.download("CADUSD=X", period="1d", auto_adjust=False, progress=False)['Close'].iloc[-1].item()
def portfolio_returns(tickers, weights, investment):
    pf = pd.DataFrame()
    for index, ticker in enumerate(tickers):
        ticker_currency = ticker_dict[ticker][1] # get the currency of the current ticker
        # ignore currencies at first and simply allocate the required amount in CAD
        allocation = investment*weights[index] # this is in CAD
        # calculate the number of shares by converting its close price to CAD
        first_close = tickers_data[ticker]["Close"].iloc[0]
        # the number of shares we buy is how many we allocate into the ticker divided by its first closing price (in CAD)
        shares = allocation/first_close.item()
        fees = min(2.15/EXCHANGE_RATE, (0.001/EXCHANGE_RATE)*shares)
        allocation_sub_fees = allocation-fees
        shares = allocation_sub_fees/first_close
        # put the initial investment in the portfolio
        pf[ticker] = shares*tickers_data[ticker]["Close"]

    pf["Total"] = pf.sum(axis=1)
    total_returns = (pf["Total"].iloc[-1] - pf["Total"].iloc[0])/(pf["Total"].iloc[0])
    total_returns = total_returns.item()
    # total portfolio value over time
    return float(total_returns), pf

# maximizing a function is the same as minimizing the negative
def negative_portfolio_returns(weights, tickers, investment):
    return -portfolio_returns(tickers, weights, investment)[0]

optimal = sp.optimize.minimize(
    fun=negative_portfolio_returns,
    x0=[1/len(top_tickers)] * len(top_tickers),
    args=(top_tickers, CAD_INITIAL_INVESTMENT),
    method='SLSQP',
    bounds=[(1/(2*len(top_tickers)), 0.15)] * len(top_tickers),
    constraints={'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    options={'ftol': 1e-12}
)
optimal_weights = optimal.x
optimal_return = optimal.fun
optimal_portfolio = portfolio_returns(top_tickers, optimal.x, CAD_INITIAL_INVESTMENT)[1]
print(f"Optimal weights: {optimal_weights}")
print(f"Optimal return: {-100*optimal_return}%")
display(optimal_portfolio)



Optimal weights: [0.15 0.13 0.02 0.02 0.02 0.15 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.15 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
Optimal return: 53.83411210376043%


,SHOP.TO,BK,AXP,CAT,LLY,C,RY.TO,AMZN,BLK,AAPL,...,TD.TO,BB.TO,QCOM,ABT,TXN,LMT,USB,UNP,PYPL,Total
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,149998.015663,129997.436929,19999.895607,19999.928540,19999.968298,149996.985700,19999.832176,19999.848539,19999.969979,19999.876044,...,19999.673536,149996.985700,19999.830860,19999.753082,19999.860913,19999.953719,19999.369321,19999.885807,19999.638053,9.999845e+05
2024-10-02,151201.054731,129704.931433,20172.642827,20084.536566,20148.982782,150530.862856,19858.582712,19959.875810,20497.388559,20050.270679,...,19845.992021,149085.150780,20306.259511,20013.842699,20181.405932,19912.145741,19904.905773,19820.697507,19991.893575,1.000716e+06
2024-10-03,149998.015663,129467.281179,19999.150282,19933.667238,20024.163433,148783.614220,19695.787742,19657.389286,20374.050708,19952.132339,...,19976.388673,143614.141258,20378.642883,19765.519583,20006.804521,19998.634010,19783.451905,19599.155201,19958.331544,9.895022e+05
2024-10-04,158390.997835,131587.904180,20548.663774,20239.484325,20060.568616,152011.177543,19894.493717,20148.930712,20324.801564,20052.039314,...,20144.042527,147261.480939,20376.229552,19837.726518,20109.979252,19975.856442,20107.328887,19454.989521,20487.559870,1.011639e+06
2024-10-07,155956.606800,132428.836558,20386.342013,20298.609149,20314.730015,151914.105603,19858.582712,19532.072498,20142.364216,19600.249492,...,20241.840016,145437.811099,20139.774624,19964.530383,19967.122864,19989.390005,20003.868565,19292.904886,20719.904061,1.006022e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-23,293641.135187,198643.254838,25425.778936,24019.893900,16890.801667,249954.524896,24543.724026,23843.604974,24233.919563,22494.886011,...,25369.244222,268535.634040,20452.233842,23932.427726,18059.397004,16065.391312,22199.029820,18568.820298,17376.733798,1.485003e+06
2025-09-24,292749.462425,197637.784631,25352.064487,23944.968652,16774.801376,246678.425603,24511.405765,23789.589288,24153.192945,22307.451157,...,25285.417295,270359.303880,20937.210353,23580.195364,18297.491488,16051.526311,22122.559819,18762.670126,17518.722462,1.481444e+06
2025-09-25,283011.919882,199411.060843,25329.725192,23635.583325,16158.396118,247794.725143,24401.280025,23567.043276,24566.675081,22710.613631,...,25604.425420,294978.868469,20470.329225,23478.047656,18059.397004,15988.475465,22055.084876,18896.246995,17229.582805,1.497672e+06


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.
- Richard Meng
- Avaansh Nanda
- Devesh Kotak